In [116]:
#importing packages
import sklearn as sk
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import seaborn as sb


In [259]:
#importing datasets
df_train = pd.read_csv('train.csv')


In [235]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [236]:
#Cleaning the data before splitting into train and test. 
df_train.info()
df_train.isnull().sum()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Things to do: 
1. Adjust Ticket to only be a numbers
2. confirm values in sex are just male and female then adjust to numerical
3. Drop name & cabin
4. Drop rows where age is null

In [185]:
#Tickets to numbers
df_train['Ticket'] = df_train['Ticket'].str.replace(r'[^0-9]', '', regex=True)
#^ Dont need this going to drop ticket

In [260]:
df_train.dropna(subset='Age', inplace=True)
df_train.dropna(subset='Embarked', inplace=True)
df_train.drop(columns=['Cabin','Name','Ticket','PassengerId'],inplace=True)
df_train.reset_index(drop=True, inplace=True)

In [261]:
#adjusting categorical columns:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [262]:
encode_data = encoder.fit_transform(df_train[['Sex','Embarked']])
encoded_df = pd.DataFrame(encode_data, columns= encoder.get_feature_names_out(['Sex','Embarked']))


In [240]:
encoded_df

,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...
707,1.0,0.0,0.0,1.0,0.0
708,0.0,1.0,0.0,0.0,1.0
709,1.0,0.0,0.0,0.0,1.0
710,0.0,1.0,1.0,0.0,0.0


In [263]:
df_train = pd.concat([df_train,encoded_df],axis=1)

In [264]:
df_train.drop(columns=['Sex','Embarked'], inplace=True,axis=1)
df_train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,1.0
1,1,1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,0.0
2,1,3,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,1.0
3,1,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0
4,0,3,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0


data is cleaned and ready for modeling

In [265]:
#looking for correlation 
corr_matrix = df_train.corr(method='pearson')
corr_matrix_abs = corr_matrix.abs()
corr_matrix_abs['Survived'].sort_values(ascending=False)


Survived      1.000000
Sex_male      0.536762
Sex_female    0.536762
Pclass        0.356462
Fare          0.266100
Embarked_C    0.195673
Embarked_S    0.159015
Parch         0.095265
Age           0.082446
Embarked_Q    0.048966
SibSp         0.015523
Name: Survived, dtype: float64

Trying to predict if someone survived or not: 
- from the above matrix it looks like the strogest correlation to survival are: Fare, and class 


In [266]:
#setting up the df for modeling
df_train_y = df_train.iloc[:,:1]
df_train_x = df_train.iloc[:,1:]
model = LinearRegression()
model.fit(df_train_x,df_train_y)

LinearRegression()

In [267]:
df_test = pd.read_csv('test.csv')

In [268]:
df_test.drop(columns=['Cabin','Name','Ticket','PassengerId'],inplace=True)


In [269]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       332 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      417 non-null    float64
 6   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 23.0+ KB


In [270]:
encode_data = encoder.fit_transform(df_test[['Sex','Embarked']])
encoded_df = pd.DataFrame(encode_data, columns= encoder.get_feature_names_out(['Sex','Embarked']))
df_test = pd.concat([df_test,encoded_df],axis=1)

In [278]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331 entries, 0 to 415
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      331 non-null    int64  
 1   Age         331 non-null    float64
 2   SibSp       331 non-null    int64  
 3   Parch       331 non-null    int64  
 4   Fare        331 non-null    float64
 5   Sex_female  331 non-null    float64
 6   Sex_male    331 non-null    float64
 7   Embarked_C  331 non-null    float64
 8   Embarked_Q  331 non-null    float64
 9   Embarked_S  331 non-null    float64
dtypes: float64(7), int64(3)
memory usage: 28.4 KB


In [272]:
df_test.drop(columns=['Embarked','Sex'],inplace=True)

In [277]:
df_test.dropna(subset=['Age','Fare'], inplace=True)

In [279]:
y_prediction = model.predict(df_test)